In [1]:
import pandas as pd
import json
import string

In [13]:
with open('../data/semEval16_plus/review_train.json') as f:
    train = json.load(f)

In [14]:
with open('../data/semEval16_plus/review_dev.json') as f:
    test = json.load(f)

In [12]:
with open('../data/yelp/inference/yelp-food-rawtext_2rest100.json') as f:
    yelp = json.load(f)

In [9]:
train[0]

{'sent_id': 'opener_en/kaf/hotel/english00192_e3fe22eeb360723a699504a27e13065e',
 'text': 'Experienced staff and had a memorable stay India as a country has always fascinated me and all of my friends who have been there always have wonderful things to say about it . One of my friends who had been there before was planning a weeklong trip to Rajasthan in India and I decided to join him this time . Yes , it really was a great experience and we visited various places but the most wonderful part of the trip was our stay at the Oberoi Udaivilas Luxury Hotel . I can ’t explain in words how grand this place looks . It is a unique blend of the old world royal charm and the modern luxuries . I ’m definitely going there again whenever I get a chance .',
 'opinions': [{'Source': [[], []],
   'Target': [['the Oberoi Udaivilas Luxury Hotel'], ['125:158']],
   'Polar_expression': [['the most wonderful part of the trip'], ['73:108']],
   'Polarity': 'Positive',
   'Intensity': 'Standard'},
  {'Source

In [15]:
# replace period with comma
for data in [train, test, yelp]:
    for i in range(len(data)):
        text = data[i]['text']
        text = text.replace('\n','')
        text = text.replace(' . ', ' , ')
        
        text_list = list(text)
        for element in range(len(text_list)):
            if element+1 < len(text_list) and element-1 >= 0:
                if text_list[element] == '.' and text_list[element+1] == '.':
                    continue

                if text_list[element] == '.' and text_list[element-1] == '.':
                    continue
                
                if text_list[element] == '.' and (text_list[element-1].isalpha() or text_list[element-1].isalnum()) and text_list[element+1] != '.':
                    text_list[element] = ' ,'

                if text_list[element] in string.punctuation and text_list[element+1]  == ' ' and text_list[element-1]  != ' ' :
                    text_list[element] = ' ' + text_list[element]
        
        if text_list[-1] in string.punctuation and text_list[-2] != ' ' and text_list[-2] != '.':
            text_list[-1] = ' ' + text_list[-1]

        text2 = "".join(text_list)
        text2 = text2.replace('  ', ' ')
        data[i]['text'] = text2


In [11]:
for i in train[20:30]:
    print(i['text'])
    print('------------')

Sensational located superfriendly hotel Top of the Tops , the staff was frienly and the rooms were new renovarted in a luxury modern style , very clean and fine , a lot of opportunitys possible and an ideal hotel for a buiseness stay or city trip
------------
Excellent hotel & service , Very happy with food Ignore the complainers ! ( If you are one · Dont expect Macdonalds or fish & chips if you go to a foreign country ! ) The five free restaurants gave an excellent choice of food although beach bar was just chips , burgers , hotdogs ( occasional BBQ fish ) and salad , All staff very friendly and service was excellent , ( These were the typical views of the vast majority of British people we spoke to at the location too , ) Booked through Flights and Packages who were excellent .
------------
Nicest hotel ever ! This was my favourite holiday place I have ever been to ! Will definitely be retuning , advise people to go here for a great holiday ! People are great ! Staff are great and th

In [16]:
def lower_exceptI (txt):
    if txt == 'i' or txt == 'I':
        return 'I'
    else:
        lst = list(txt.lower())
        for i in range(len(lst)):
            if i+1 < len(lst) and i-1 >= 0:
                if lst[i] == 'i' and (lst[i+1] == ' ' or lst[i+1] in string.punctuation) and lst[i-1] == ' ':
                    lst[i] = lst[i].upper()
                elif lst[i] == 'i' and (lst[i+1].isalpha() == False) and (lst[i-1].isalpha() == False):
                    lst[i] = lst[i].upper()
            elif i == 0 and lst[i] == 'i' and (lst[i+1] == ' ' or lst[i+1] in string.punctuation):
                lst[i] = lst[i].upper()
            else:
                lst[i] = lst[i].lower()
        return ("".join(lst))

In [17]:
# convert uppercase to lowercase for all letters (also for targets and expression)
# only I be uppercase
for data in [train, test, yelp]:
    for i in range(len(data)):
        data[i]['text'] = lower_exceptI(data[i]['text'])

        if len(data[i]['opinions']) > 0:
            for op in range(len(data[i]['opinions'])):
                source = data[i]['opinions'][op]['Source'][0]
                target = data[i]['opinions'][op]['Target'][0]
                exp = data[i]['opinions'][op]['Polar_expression'][0]

                source_lower = [lower_exceptI(s) for s in source]
                target_lower = [lower_exceptI(t) for t in target]
                exp_lower = [lower_exceptI(e) for e in exp]

                data[i]['opinions'][op]['Source'][0] = source_lower
                data[i]['opinions'][op]['Target'][0] = target_lower
                data[i]['opinions'][op]['Polar_expression'][0] = exp_lower 

In [18]:
# replace , ! with and
for data in [train, test, yelp]:
    for i in range(len(data)):
        text = data[i]['text']
        text = text.replace(' , ', ' and ')
        text = text.replace(' ! ', ' and ')
        text = text.replace(' and !', ' ! !')
        text = text.replace('and but', 'but')
        data[i]['text'] = text

In [15]:
for i in train[20:30]:
    print(i['text'])
    print('------------')

sensational located superfriendly hotel top of the tops and the staff was frienly and the rooms were new renovarted in a luxury modern style and very clean and fine and a lot of opportunitys possible and an ideal hotel for a buiseness stay or city trip
------------
excellent hotel & service and very happy with food ignore the complainers and ( if you are one · dont expect macdonalds or fish & chips if you go to a foreign country and ) the five free restaurants gave an excellent choice of food although beach bar was just chips and burgers and hotdogs ( occasional bbq fish ) and salad and all staff very friendly and service was excellent and ( these were the typical views of the vast majority of british people we spoke to at the location too and ) booked through flights and packages who were excellent .
------------
nicest hotel ever and this was my favourite holiday place I have ever been to and will definitely be retuning and advise people to go here for a great holiday and people are 

In [19]:
train[-7]

{'sent_id': 'TFS#2',
 'text': 'the four seasons restaurant is a great experience and the food is great and the environment is even better .',
 'opinions': [{'Source': [[], []],
   'Target': [['the four seasons restaurant'], ['0:27']],
   'Polar_expression': [['great experience.'], ['33:50']],
   'Polarity': 'Positive',
   'Intensity': 'Standard'},
  {'Source': [[], []],
   'Target': [['the food'], ['0:8']],
   'Polar_expression': [['great'], ['12:17']],
   'Polarity': 'Positive',
   'Intensity': 'Standard'},
  {'Source': [[], []],
   'Target': [['the environment'], ['22:37']],
   'Polar_expression': [['even better.'], ['41:53']],
   'Polarity': 'Negative',
   'Intensity': 'Standard'}],
 'dataset': 's16+_train'}

In [20]:
yelp[1]

{'review_id': 'gWLiqQWybWwIZS4gT9Spbw',
 'user_id': 'ryNLy69uEwBSGQfSl5ZXzg',
 'business_id': '_ab50qdWOk0DdB6XOrBitw',
 'stars': 3.0,
 'useful': 0,
 'funny': 0,
 'cool': 0,
 'text': 'acme has a welcoming atmosphere to a city guy and old and not much natural light and bustling and and kind of dirty like every bar should be ,I ate here twice for quick lunches and my first experience was better and seated at the oyster bar and the shucker was quite friendly and after serving me 10 oysters (I ordered the half dozen ) he asked how I enjoyed them and "I dont touch \'em, " he replied to my favorable review and it was pretty funny ,I also had the seafood gumbo and their signature po\'boy and which were both decent ,go for oysters and beer .',
 'date': '2008-09-30 15:00:00',
 'sent_id': 'gWLiqQWybWwIZS4gT9Spbw',
 'opinions': []}

In [48]:
# output json 
with open('../data/semEval16_plus/noperiod_dev.json', 'w') as outfile1:
    json.dump(test, outfile1, ensure_ascii = False, indent = 4)

In [47]:
# output json 
with open('../data/semEval16_plus/noperiod_train.json', 'w') as outfile2:
    json.dump(train, outfile2, ensure_ascii = False, indent = 4)

In [21]:
# output json 
with open('../data/yelp/inference/yelp-food-nopr_2rest100.json', 'w') as outfile2:
    json.dump(yelp, outfile2, ensure_ascii = False, indent = 4)